In [212]:
# Import libraries
import numpy as np
import pandas as pd
import datetime as dt

from sklearn.model_selection import train_test_split

In [213]:
# Get data
train_path = '../data/train.csv'
store_path = '../data/store.csv'

In [214]:
train = pd.read_csv(train_path)
store = pd.read_csv(store_path)

c:\users\sheon\appdata\local\programs\python\python38\lib\site-packages\IPython\core\interactiveshell.py:3145: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


## Prepare train dataset

In [215]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 914629 entries, 0 to 914628
Data columns (total 9 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   Store          914629 non-null  int64 
 1   DayOfWeek      914629 non-null  int64 
 2   Date           914629 non-null  object
 3   Sales          914629 non-null  int64 
 4   Customers      914629 non-null  int64 
 5   Open           914629 non-null  int64 
 6   Promo          914629 non-null  int64 
 7   StateHoliday   914629 non-null  object
 8   SchoolHoliday  914629 non-null  int64 
dtypes: int64(7), object(2)
memory usage: 62.8+ MB


In [216]:
# Drop records with Open==0 and Customers==0
# No customers or when closed would mean 0 sales

train = train.drop(train[train['Open']==0].index)
train = train.drop(train[train['Customers']==0].index)

In [217]:
# Change datatypes
# Date to datetime object
# Sales to float

train['Date'] = train['Date'].apply(lambda x: dt.datetime.strptime(x, '%Y-%m-%d'))
train['Sales'] = train['Sales'].astype(float)

In [218]:
# Clean fields

train['StateHoliday'] = train['StateHoliday'].replace(0, '0')

## Prepare store dataset

In [219]:
store.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1115 entries, 0 to 1114
Data columns (total 10 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Store                      1115 non-null   int64  
 1   StoreType                  1115 non-null   object 
 2   Assortment                 1115 non-null   object 
 3   CompetitionDistance        1112 non-null   float64
 4   CompetitionOpenSinceMonth  761 non-null    float64
 5   CompetitionOpenSinceYear   761 non-null    float64
 6   Promo2                     1115 non-null   int64  
 7   Promo2SinceWeek            571 non-null    float64
 8   Promo2SinceYear            571 non-null    float64
 9   PromoInterval              571 non-null    object 
dtypes: float64(5), int64(2), object(3)
memory usage: 87.2+ KB


In [220]:
# Clean change datatype of 'CompetitionOpenSinceMonth', 'CompetitionOpenSinceYear'

store['CompetitionOpenSinceMonth'] = store['CompetitionOpenSinceMonth'].fillna(0)
store['CompetitionOpenSinceYear'] = store['CompetitionOpenSinceYear'].fillna(0)

store['CompetitionOpenSinceMonth'] = store['CompetitionOpenSinceMonth'].astype(int)
store['CompetitionOpenSinceYear'] = store['CompetitionOpenSinceYear'].astype(int)

store['CompetitionOpenSinceMonth'] = store['CompetitionOpenSinceMonth'].astype(str)
store['CompetitionOpenSinceYear'] = store['CompetitionOpenSinceYear'].astype(str)

store['CompetitionOpenSinceMonth'] = store['CompetitionOpenSinceMonth'].str.pad(width=2, side='left', fillchar='0')

store['CompetitionOpenSinceMonth'] = store['CompetitionOpenSinceMonth'].replace('00', '01')
store['CompetitionOpenSinceYear'] = store['CompetitionOpenSinceYear'].replace('0', '1800')

store['CompetitionOpenSince'] = store['CompetitionOpenSinceMonth']+'_'+store['CompetitionOpenSinceYear']

store['CompetitionOpenSince'] = store['CompetitionOpenSince'].apply(lambda x: dt.datetime.strptime(x, '%m_%Y'))

In [221]:
# Clean change datatype of 'Promo2SinceMonth', 'Promo2SinceYear'

store['Promo2SinceWeek'] = store['Promo2SinceWeek'].fillna(0)
store['Promo2SinceYear'] = store['Promo2SinceYear'].fillna(0)

store['Promo2SinceWeek'] = store['Promo2SinceWeek'].astype(int)
store['Promo2SinceYear'] = store['Promo2SinceYear'].astype(int)

store['Promo2SinceWeek'] = store['Promo2SinceWeek'].astype(str)
store['Promo2SinceYear'] = store['Promo2SinceYear'].astype(str)

store['Promo2SinceWeek'] = store['Promo2SinceWeek'].str.pad(width=2, side='left', fillchar='0')

store['Promo2SinceWeek'] = store['Promo2SinceWeek'].replace('00', '01')
store['Promo2SinceYear'] = store['Promo2SinceYear'].replace('0', '1800')

store['Promo2Since'] = store['Promo2SinceWeek']+'_'+store['Promo2SinceYear']+' SUN'

store['Promo2Since'] = store['Promo2Since'].apply(lambda x: dt.datetime.strptime(x, '%U_%Y %a'))

In [222]:
# Clean 'PromoInterval'. Replace all Nan to 'None'
store['PromoInterval'] = store['PromoInterval'].fillna('None')

In [223]:
store['CompetitionDistance'] = store['CompetitionDistance'].fillna(0)

## Join train and store

In [224]:
data = pd.merge(train, store, on='Store', how='left')

In [225]:
# Create 'DaysSinceCompetitionOpen' feature

data['DaysSinceCompetitionOpen'] = data.apply(lambda x: 0 if x['CompetitionOpenSince'].year<=dt.date(1800, 12, 31).year else (0 if (x['Date'] <= x['CompetitionOpenSince']) else (x['Date']-x['CompetitionOpenSince']).days), axis=1)

In [226]:
# Create 'DaysSincePromo2' feature

data['DaysSincePromo2'] = data.apply(lambda x: 0 if x['Promo2Since'].year<=dt.date(1800, 12, 31).year else (0 if (x['Date'] <= x['Promo2Since']) else (x['Date']-x['Promo2Since']).days), axis=1)

In [227]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 759850 entries, 0 to 759849
Data columns (total 22 columns):
 #   Column                     Non-Null Count   Dtype         
---  ------                     --------------   -----         
 0   Store                      759850 non-null  int64         
 1   DayOfWeek                  759850 non-null  int64         
 2   Date                       759850 non-null  datetime64[ns]
 3   Sales                      759850 non-null  float64       
 4   Customers                  759850 non-null  int64         
 5   Open                       759850 non-null  int64         
 6   Promo                      759850 non-null  int64         
 7   StateHoliday               759850 non-null  object        
 8   SchoolHoliday              759850 non-null  int64         
 9   StoreType                  759850 non-null  object        
 10  Assortment                 759850 non-null  object        
 11  CompetitionDistance        759850 non-null  float64 

In [228]:
data.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,...,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,CompetitionOpenSince,Promo2Since,DaysSinceCompetitionOpen,DaysSincePromo2
0,1,4,2015-04-30,6228.0,650,1,1,0,0,c,...,09,2008,0,01,1800,None,2008-09-01,1800-01-05,2432,0
1,2,4,2015-04-30,6884.0,716,1,1,0,0,a,...,11,2007,1,13,2010,"Jan,Apr,Jul,Oct",2007-11-01,2010-03-28,2737,1859
2,3,4,2015-04-30,9971.0,979,1,1,0,0,a,...,12,2006,1,14,2011,"Jan,Apr,Jul,Oct",2006-12-01,2011-04-03,3072,1488
3,4,4,2015-04-30,16106.0,1854,1,1,0,0,c,...,09,2009,0,01,1800,None,2009-09-01,1800-01-05,2067,0
4,5,4,2015-04-30,6598.0,729,1,1,0,0,a,...,04,2015,0,01,1800,None,2015-04-01,1800-01-05,29,0


In [229]:
for_model = data[['Store', 'Date', 'Sales', 
                  'Customers', 'Promo', 'StateHoliday', 'StoreType']]

In [230]:
for_model

,Store,Date,Sales,Customers,Promo,StateHoliday,StoreType
0,1,2015-04-30,6228.0,650,1,0,c
1,2,2015-04-30,6884.0,716,1,0,a
2,3,2015-04-30,9971.0,979,1,0,a
3,4,2015-04-30,16106.0,1854,1,0,c
4,5,2015-04-30,6598.0,729,1,0,a
...,...,...,...,...,...,...,...
759845,682,2013-01-01,3375.0,566,0,a,b
759846,733,2013-01-01,10765.0,2377,0,a,b
759847,769,2013-01-01,5035.0,1248,0,a,b
759848,948,2013-01-01,4491.0,1039,0,a,b


In [256]:
new_df = pd.DataFrame()
store1 = for_model[for_model['Store']==1]
store1 = store1.sort_values('Date', ascending=True)

In [251]:
store1['prev_sales'] = store1['Sales'].shift(1)

In [253]:
store1 = store1.dropna()

In [257]:
new_df = new_df.append(store1)

In [258]:
new_df

,Store,Date,Sales,Customers,Promo,StateHoliday,StoreType
758722,1,2013-01-02,5530.0,668,0,0,c
757613,1,2013-01-03,4327.0,578,0,0,c
756505,1,2013-01-04,4486.0,619,0,0,c
755398,1,2013-01-05,4997.0,635,0,0,c
754273,1,2013-01-07,7176.0,785,1,0,c
...,...,...,...,...,...,...,...
4491,1,2015-04-25,4318.0,537,0,0,c
3344,1,2015-04-27,5575.0,574,1,0,c
2230,1,2015-04-28,5199.0,552,1,0,c
1115,1,2015-04-29,5775.0,579,1,0,c


In [259]:
new_df = pd.DataFrame()

for store in for_model['Store'].unique():
    curr_store = for_model[for_model['Store']==store]
    curr_store = curr_store.sort_values('Date', ascending=True)
    curr_store['prev_sales'] = curr_store['Sales'].shift(1)
    curr_store = curr_store.dropna()
    
    new_df = new_df.append(curr_store)

In [260]:
new_df

,Store,Date,Sales,Customers,Promo,StateHoliday,StoreType,prev_sales
757613,1,2013-01-03,4327.0,578,0,0,c,5530.0
756505,1,2013-01-04,4486.0,619,0,0,c,4327.0
755398,1,2013-01-05,4997.0,635,0,0,c,4486.0
754273,1,2013-01-07,7176.0,785,1,0,c,4997.0
753168,1,2013-01-08,5580.0,654,1,0,c,7176.0
...,...,...,...,...,...,...,...,...
5605,1115,2015-04-25,7620.0,549,0,0,d,5245.0
4458,1115,2015-04-27,10211.0,578,1,0,d,7620.0
3343,1115,2015-04-28,7287.0,481,1,0,d,10211.0
2229,1115,2015-04-29,8148.0,518,1,0,d,7287.0


In [261]:
new_df = pd.get_dummies(new_df, 
                           columns=['StateHoliday', 'StoreType'],
                           drop_first=True)

In [264]:
new_df = new_df.sort_index()

In [265]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 758735 entries, 0 to 759814
Data columns (total 12 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   Store           758735 non-null  int64         
 1   Date            758735 non-null  datetime64[ns]
 2   Sales           758735 non-null  float64       
 3   Customers       758735 non-null  int64         
 4   Promo           758735 non-null  int64         
 5   prev_sales      758735 non-null  float64       
 6   StateHoliday_a  758735 non-null  uint8         
 7   StateHoliday_b  758735 non-null  uint8         
 8   StateHoliday_c  758735 non-null  uint8         
 9   StoreType_b     758735 non-null  uint8         
 10  StoreType_c     758735 non-null  uint8         
 11  StoreType_d     758735 non-null  uint8         
dtypes: datetime64[ns](1), float64(2), int64(3), uint8(6)
memory usage: 44.9 MB


In [275]:
X = new_df.drop(['Store', 'Date', 'Sales'], axis=1)
y = new_df[['Sales']]

In [276]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [295]:
stores = list(new_df['Store'].unique())

In [303]:
import random

train_prop = int(0.7*len(stores))
print(train_prop)

train_list = random.sample(stores, train_prop)
test_list = []

for store in stores:
    if store not in train_list:
        test_list.append(store)

print(len(test_list))
print(str(len(test_list)+len(train_list)))

780
335
1115


In [299]:
X = pd.DataFrame()


ValueError: ('Lengths must match to compare', (758735,), (780,))

## Linear Regression

In [277]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train)
print(lin_reg.score(X_train, y_train))
lin_pred = lin_reg.predict(X_test)
print(lin_reg.score(X_test, y_test))
print(mean_squared_error(y_test, lin_pred))

0.8379828356118637
0.8386863885732755
1561751.859706707


## Decision Tree Regression

In [278]:
from sklearn.tree import DecisionTreeRegressor

tree_reg = DecisionTreeRegressor()
tree_reg.fit(X_train, y_train)
print(tree_reg.score(X_train, y_train))
tree_pred = tree_reg.predict(X_test)
print(tree_reg.score(X_test, y_test))
print(mean_squared_error(y_test, tree_pred))

0.9991613849231343
0.7630642645559487
2293884.7019044575


## Random Forest Regression

In [279]:
from sklearn.ensemble import RandomForestRegressor

forest_reg = RandomForestRegressor()
forest_reg.fit(X_train, y_train)
print(forest_reg.score(X_train, y_train))
forest_pred = tree_reg.predict(X_test)
print(forest_reg.score(X_test, y_test))
print(mean_squared_error(y_test, forest_pred))

<ipython-input-279-7ebbab1b78d4>:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  forest_reg.fit(X_train, y_train)


0.9801889084879774
0.8604698391153494
2293884.7019044575


## Gradient Boosting

In [280]:
from sklearn.ensemble import GradientBoostingRegressor

gb_reg = GradientBoostingRegressor()
gb_reg.fit(X_train, y_train)
print("train r2: " + str(gb_reg.score(X_train, y_train)))
gb_pred = gb_reg.predict(X_test)
print("test r2: "+ str(gb_reg.score(X_test, y_test)))
print("mse: " + str(mean_squared_error(y_test, gb_pred)))

c:\users\sheon\appdata\local\programs\python\python38\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


train r2: 0.870451667080314
test r2: 0.8699319022940883
mse: 1259249.5554722194


## XGBoost

In [281]:
import xgboost as xgb

param = {'booster': 'gbtree',
         'eta': 0.3,
         'max_depth': 6,
         'subsample': 0.7,
         'colsample_bytree': 0.7,
         'eval_metric':'rmse'}

In [282]:
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

In [283]:
xgb_reg = xgb.train(param, dtrain)

In [284]:
xgb_pred = xgb_reg.predict(dtest)

In [285]:
mean_squared_error(y_test, xgb_pred)

1317875.2756795709

## LSTM

In [288]:
import keras